In [80]:
#!/usr/bin/env python
# coding: utf-8

# In[313]:


#import all modules here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')


In [81]:
def Approach3(data_df,label_column, N, noise_frac):

    def add_noise(data, col):
        class0 = data.loc[data[col]!=1,:]
        class1 = data.loc[data[col]!=0,:]
        def noise(data):
            sample = data.sample(frac = noise_frac)
            for i in list(sample.index):
                data[col][i] = int(not data[col][i])
            return list(sample.index)    
        noise0 = noise(class0)
        noise1 = noise(class1)
        noisy_indices = noise0+noise1
        mislabeled = pd.concat([class0,class1])
        indices = list(mislabeled.index)
        return mislabeled,noisy_indices


    # In[317]:


    mislabeled,noisy_indices = add_noise(data_df, label_column)


    # In[318]:


    def distance_from_centroid(df,centroids,data_label):
        d = defaultdict(list)
        for index in df.index:
            dist = 0
            for i in range(len(centroids[0])):
                dist+=(centroids[df.loc[index,'cluster']][i] - df.loc[index,df.columns[i]])**2
            d[df.loc[index,'cluster']].append([dist,index,data_label[index]])
        return d


    # In[ ]:





    # In[319]:


    def ratio_in_cluster(features,data_label,n):
        majority_label_in_cluster = defaultdict(None)
        km = KMeans(n_clusters = n)
        km.fit(features)
        label = km.labels_
        print(len(label))
        print(features.shape)
        features['cluster'] = label

        distance = distance_from_centroid(features,km.cluster_centers_,data_label)

        for i in distance.keys():
            distance[i] = sorted(distance[i],key=lambda x: x[0])

        normalized_distance = dict(distance)
        label_ratio_count = {}

        for i in normalized_distance:
            temp = [0,0]
            for j in range(len(normalized_distance[i])):
                normalized_distance[i][j][0] = normalized_distance[i][j][0]/normalized_distance[i][-1][0]
                temp[normalized_distance[i][j][-1]] +=1

            temp[0],temp[1] = temp[0]/sum(temp),temp[1]/sum(temp)
            label_ratio_count[i] = temp

            if temp[0] > temp[1] and temp[0]>=0.7:
                majority_label_in_cluster[i] = 0
            elif temp[0] < temp[1] and temp[1]>=0.7:
                majority_label_in_cluster[i] = 1
            else:
                majority_label_in_cluster[i] = None

        return(normalized_distance, majority_label_in_cluster)
        #return pd.DataFrame(data=label_ratio_count),majority_label_in_cluster  



    # In[320]:


    normalized_distance, majority_label_in_cluster = ratio_in_cluster(mislabeled.loc[:,mislabeled.columns!=label_column],mislabeled[label_column],N)


    # In[321]:


    points_removed = []
    reduced_points = defaultdict(list)
    for cluster_id in normalized_distance.keys():
        count = 0
        for point in normalized_distance[cluster_id]:
            if count/len(normalized_distance[cluster_id]) > 0.75:
                if point[-1] != majority_label_in_cluster[cluster_id] and majority_label_in_cluster[cluster_id]!=None:
                    points_removed.append(point)
                    continue
                else:
                    reduced_points[cluster_id].append(point)
                    count+=1
            else:
                reduced_points[cluster_id].append(point)
                count +=1


    # In[322]:


    # result = []
    # for cluster_id in normalized_distance:
    #     print(len(normalized_distance[cluster_id]), len(reduced_points[cluster_id]))
    #     print((len(reduced_points[cluster_id]) * 1.0) -  len(normalized_distance[cluster_id]))
    #     print()


    # In[323]:


    #reduced_points


    # In[324]:


    #majority_label_in_cluster


    # In[325]:


    #points_removed


    # In[326]:


    noisy_indices = set(noisy_indices)
    n_n = 0
    p_n = 0
    for point in points_removed:
        if point[2] in noisy_indices:
            n_n+=1
        else:
            p_n+=1
    p_p = (len(mislabeled) - len(noisy_indices)) - p_n
    n_p = len(noisy_indices) - n_n
    print("# Observation in Culprit",len(points_removed))
    print("Noise to Noise",n_n)
    print("Pure to Noise",p_n)
    print("Noise to Pure",n_p)
    print("Pure to Pure",p_p)
    print('Pure %',100 * p_p/(len(mislabeled)-len(points_removed)))
    print('Noisy %',100 * n_p/(len(mislabeled)-len(points_removed)))
    print('Cleaned %',100 * n_n/(len(noisy_indices)))


    # # 

In [82]:
# N = 14
# data_df = pd.read_csv("../haberman.data",header=None)
# data_df.fillna(data_df.median(),inplace=True)
# data_df.columns = ['Age','Year','Nodes','Survival_status']
# data_df['Survival_status'] = data_df['Survival_status'].astype(int)
# data_df['Survival_status'] = data_df['Survival_status'].map({2:0 , 1:1})
# label_column = 'Survival_status'
# noise_frac = 0.3

In [83]:
# N = 14
# data_df = pd.read_csv('../SPECT.train',header=None)
# label_column = 0

In [84]:
data_df = pd.read_csv('../Binary Classified Datasets/ionosphere.csv')
label_column = 'g'
N = 2

In [85]:
for i in range(1,6):
    print(i/10)
    Approach3(data_df,label_column,N,i/10)
    print()

0.1
350
(350, 34)
# Observation in Culprit 24
Noise to Noise 0
Pure to Noise 24
Noise to Pure 35
Pure to Pure 291
Pure % 89.2638036809816
Noisy % 10.736196319018404
Cleaned % 0.0

0.2
350
(350, 34)
# Observation in Culprit 24
Noise to Noise 0
Pure to Noise 24
Noise to Pure 70
Pure to Pure 256
Pure % 78.52760736196319
Noisy % 21.47239263803681
Cleaned % 0.0

0.3
350
(350, 34)
# Observation in Culprit 0
Noise to Noise 0
Pure to Noise 0
Noise to Pure 105
Pure to Pure 245
Pure % 70.0
Noisy % 30.0
Cleaned % 0.0

0.4
350
(350, 34)
# Observation in Culprit 0
Noise to Noise 0
Pure to Noise 0
Noise to Pure 140
Pure to Pure 210
Pure % 60.0
Noisy % 40.0
Cleaned % 0.0

0.5
350
(350, 34)
# Observation in Culprit 0
Noise to Noise 0
Pure to Noise 0
Noise to Pure 175
Pure to Pure 175
Pure % 50.0
Noisy % 50.0
Cleaned % 0.0



In [86]:
print(len(data_df))

350


In [87]:
data_df.head()

,X1,X0,X0.99539,X.0.05889,X0.85243,X0.02306,X0.83398,X.0.37708,X1.1,X0.03760,...,X.0.51171,X0.41078,X.0.46168,X0.21266,X.0.34090,X0.42267,X.0.54487,X0.18641,X.0.45300,g
0,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
1,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
2,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
3,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1
4,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,...,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011,0
